In [1]:
# use the code from: 
# https://htmlpreview.github.io/?https://github.com/jinworks/CellChat/blob/master/tutorial/Update-CellChatDB.html

In [3]:
library(CellChat)
options(stringsAsFactors = FALSE)

In [4]:
getwd()

[1] "/home/tanasa/Desktop/dyslexia_results_p51_NEURONCHAT/cellphonedb-data-5.0.0_to_cell_neuronchat_transform_into_CellChat/cellphonedb"

In [5]:
# CellChatDB.human

In [6]:
dim(CellChatDB.human$interaction)

[1] 3233   28

In [7]:
write.csv(CellChatDB.human$interaction, file = "CellChat_Interactions_before_merging_with_CellPhoneDB.csv", row.names = FALSE)

In [8]:
list.files()

[1] "CellChat_Interactions_before_merging_with_CellPhoneDB.csv"
 [2] "CellChat_Interactions_from_CellPhoneDB.csv"               
 [3] "CellChat_Interactions_native_CellChat.csv"                
 [4] "cellphonedb-data"                                         
 [5] "CellPhoneDB.to.Cell.Chat.human.rda"                       
 [6] "CellPhoneDB.to.Cell.Chat.ipynb"                           
 [7] "common_interactions.txt"                                  
 [8] "complex_composition_table.csv"                            
 [9] "complex_table.csv"                                        
[10] "gene_search_results.txt"                                  
[11] "gene_synonym_to_gene_name.csv"                            
[12] "gene_table.csv"                                           
[13] "genes_found.txt"                                          
[14] "genes_not_found.txt"                                      
[15] "interaction_table.csv"                                    
[16] "multidata_table.csv"                                      
[17] "NeuronChat_LR_genes_list.txt"                             
[18] "protein_table.csv"                                        
[19] "receptor_to_transcription_factor.csv"                     
[20] "search_genes.sh"                                          
[21] "unique_to_file1.txt"                                      
[22] "unique_to_file2.txt"

In [9]:
cat("Step 1: Load the database files")

Step 1: Load the database files

In [10]:
interaction_input <- read.csv(file = './cellphonedb-data/data/interaction_input.csv')
complex_input <- read.csv(file = './cellphonedb-data/data/complex_input.csv', row.names = 1)
geneInfo <- read.csv(file = './cellphonedb-data/data/gene_input.csv')
geneInfo$Symbol <- geneInfo$hgnc_symbol
geneInfo <- select(geneInfo, -c("ensembl"))
geneInfo <- unique(geneInfo)

In [11]:
CellChatDB <- CellChatDB.human 

In [12]:
geneInfo$Symbol <- geneInfo$hgnc_symbol
# geneInfo <- select(geneInfo, -c("ensembl"))
geneInfo <- unique(geneInfo)

In [13]:
head(geneInfo)

,gene_name,uniprot,hgnc_symbol,Symbol
,<chr>,<chr>,<chr>,<chr>
1,SULT1A1,P50225,SULT1A1,SULT1A1
2,UBASH3B,Q8TF42,UBASH3B,UBASH3B
3,SRD5A3,Q9H8P0,SRD5A3,SRD5A3
4,SULT2A1,Q06520,SULT2A1,SULT2A1
5,SRD5A2,P31213,SRD5A2,SRD5A2
6,SULT1E1,P49888,SULT1E1,SULT1E1


In [14]:
cat("Step 2: Formulate the input files to be compatible with CellChatDB")

Step 2: Formulate the input files to be compatible with CellChatDB

In [15]:
## prepare interaction_input file

# get the ligand information
idx_partnerA <- match(interaction_input$partner_a, geneInfo$uniprot)
idx.use <- !is.na(idx_partnerA)
interaction_input$ligand <- interaction_input$partner_a
interaction_input$ligand[idx.use] <- geneInfo$hgnc_symbol[idx_partnerA[idx.use]]

# get the receptor information
idx_partnerB <- match(interaction_input$partner_b, geneInfo$uniprot)
idx.use <- !is.na(idx_partnerB)
interaction_input$receptor <- interaction_input$partner_b
interaction_input$receptor[idx.use] <- geneInfo$hgnc_symbol[idx_partnerB[idx.use]]

# get other information
interaction_input$interaction_name <- interaction_input$interactors
interaction_input$interaction_name_2 <- interaction_input$interaction_name
interaction_input$pathway_name <- interaction_input$classification
interaction_input$pathway_name <- gsub(".*by ", "", interaction_input$pathway_name)
interaction_input$annotation <- interaction_input$directionality
interaction_input <- select(interaction_input, -c("partner_a","partner_b","protein_name_a","protein_name_b","interactors","classification","directionality"))

## prepare complex_input file

complexsubunits <- dplyr::select(complex_input, starts_with("uniprot"))
for (i in 1:ncol(complexsubunits)) {
  idx_complex <- match(complex_input[,paste0("uniprot_",i)], geneInfo$uniprot)
  idx.use <- !is.na(idx_complex)
  complex_input[idx.use,paste0("uniprot_",i)] <- geneInfo$hgnc_symbol[idx_complex[idx.use]]
}
colnames(complex_input)[1:ncol(complexsubunits)] <- paste0("subunit_",seq_len(ncol(complexsubunits)))
complex_input <- dplyr::select(complex_input, starts_with("subunit"))

## prepare other information 
other_info <- list(complex = complex_input)

In [16]:
cat("Step 3: Update CellChatDB by running the function updateCellChatDB")

Step 3: Update CellChatDB by running the function updateCellChatDB

In [17]:
db.new <- updateCellChatDB(db = interaction_input, 
                           gene_info = geneInfo, 
                           other_info = other_info, 
                           trim.pathway = T)

The pathway names of 136 interactions are missing and the corresponding interactions are now deleted. 


Warning message in updateCellChatDB(db = interaction_input, gene_info = geneInfo, :
“2 duplicated interaction_names are identified and the corresponding interactions are now deleted. 
”


In [18]:
dim(db.new$interaction)

[1] 2773   22

In [19]:
head(db.new$interaction)

,interaction_name,pathway_name,ligand,receptor,agonist,antagonist,co_A_receptor,co_I_receptor,annotation,interaction_name_2,⋯,source,is_ppi,curator,reactome_complex,reactome_reaction,reactome_pathway,complexPortal_complex,comments,version,modulatory_effect
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CDH1-ITGA2+ITGB1,CDH1-ITGA2+ITGB1,Cadherin,CDH1,integrin_a2b1_complex,,,,,Adhesion-Adhesion,CDH1-ITGA2+ITGB1,⋯,PMID:12392763,True,RVentoTormo,,,,NA,,CellPhoneDBcore<=4.1,
CDH1-ITGAE+ITGB7,CDH1-ITGAE+ITGB7,Cadherin,CDH1,integrin_aEb7_complex,,,,,Adhesion-Adhesion,CDH1-ITGAE+ITGB7,⋯,uniprot,True,RVentoTormo,,,,NA,,CellPhoneDBcore<=4.1,
CDH1-KLRG1,CDH1-KLRG1,Cadherin,CDH1,KLRG1,,,,,Ligand-Receptor,CDH1-KLRG1,⋯,PMC3030123,True,RVentoTormo,,,,NA,,CellPhoneDBcore<=4.1,
CDH2-CDH2,CDH2-CDH2,Cadherin,CDH2,CDH2,,,,,Adhesion-Adhesion,CDH2-CDH2,⋯,uniprot,True,LGarciaAlonso,,,,NA,,CellPhoneDBcore<=4.1,
CDH2-FCER2,CDH2-FCER2,Cadherin,CDH2,FCER2,,,,,Ligand-Receptor,CDH2-FCER2,⋯,Shilts screen - VALIDATED,True,JShilts,,,,NA,,CellPhoneDBcore4.1,
CDH3-CDH3,CDH3-CDH3,Cadherin,CDH3,CDH3,,,,,Adhesion-Adhesion,CDH3-CDH3,⋯,uniprot,True,LGarciaAlonso,,,,NA,,CellPhoneDBcore<=4.1,


In [20]:
# ?updateCellChatDB
# Update CellChatDB by integrating new L-R pairs from other resources or adding more information 

In [21]:
write.csv(db.new$interaction, 
          file = "CellChat_Interactions_is_it_from_CellPhoneDB.csv", 
          row.names = FALSE)

In [22]:
getwd()

[1] "/home/tanasa/Desktop/dyslexia_results_p51_NEURONCHAT/cellphonedb-data-5.0.0_to_cell_neuronchat_transform_into_CellChat/cellphonedb"

In [23]:
save(db.new, file = "CellPhoneDB.to.Cell.Chat.human.rda")